# CityBikes

Send a request to CityBikes for the city of your choice. 

## City Bike Approach

### Sandbox and POC 
* GET the list of ALL providers using the general link
* In the Main list there are several providers for each city (Vancovuer x3). 
* There are not specific details in the main list. 
* GET requests are required for each provider in the target city. 
* Each **detailed** request will have to be consolidated into a DataFrame


## Step 1 - Obtain Master List of City Bike providers

* Library required:
    * requests - for API GET
    * json - to import the json response 
    * pandas to manipulate the dataframes

In [2]:
import requests
import json
import pandas as pd

In [ ]:
## Create a funtion to pull City Bike info. 
'''City is the only parameter for the call. The default will be null'''

provider_id =''
url = f"http://api.citybik.es/v2/networks/{provider_id}"
payload = {}
headers = {}
print(url)

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

if response.status_code == 200:
    # Construct the filename with the index
    filename = f'output_citybike_ALL.json'
    with open(filename, 'w') as json_file:
        json.dump(response.json(), json_file, indent=4)
        print('JSON data saved successfully.')
else:
    print(f"Error: Received status code {response.status_code}")




## Step 2 - Parse list to identify provider ID for Target City

* create an object within Python using the json response.
* Dont forget to NORMALIZE (ie flatten it). 

Json structure: 
* networks
    * company
    * href
    * id
    * location:
        * city: **Target City**
    * name

In [4]:
city_bike_json= response.json()

# Use pd.json_normalize to flatten the "networks" JSON array
city_bike_df = pd.json_normalize(city_bike_json['networks'], errors='ignore')
city_bike_df

,company,href,id,name,location.city,location.country,location.latitude,location.longitude,source,gbfs_href,license.name,license.url,ebikes
0,[ЗАО «СитиБайк»],/v2/networks/velobike-moscow,velobike-moscow,Velobike,Moscow,RU,55.750000,37.616667,NaN,NaN,NaN,NaN,NaN
1,[Urban Infrastructure Partner],/v2/networks/baerum-bysykkel,baerum-bysykkel,Bysykkel,Bærum,NO,59.894550,10.546343,NaN,NaN,NaN,NaN,NaN
2,[Comunicare S.r.l.],/v2/networks/bicincitta-siena,bicincitta-siena,Bicincittà,Siena,IT,43.318600,11.330600,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN,NaN
3,[Cyclopolis Systems],/v2/networks/cyclopolis-maroussi,cyclopolis-maroussi,Cyclopolis,Maroussi,GR,38.056872,23.808330,NaN,NaN,NaN,NaN,NaN
4,[Cyclopolis Systems],/v2/networks/cyclopolis-nafplio,cyclopolis-nafplio,Cyclopolis,Nafplio,GR,37.563940,22.809340,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,[Instituto Tecnológico de Castilla y León (ITCL)],/v2/networks/albabici,albabici,AlbaBici,Albacete,ES,38.994300,-1.860200,NaN,NaN,NaN,NaN,NaN
586,[Instituto Tecnológico de Castilla y León (ITCL)],/v2/networks/bicialcazar,bicialcazar,BiciAlcázar,Alcázar de San Juan,ES,39.390100,-3.210100,NaN,NaN,NaN,NaN,NaN
587,[Instituto Tecnológico de Castilla y León (ITCL)],/v2/networks/tallinja-bike,tallinja-bike,Tallinja Bike,Valletta - Malta - Gozo,MT,35.899200,14.514100,NaN,NaN,NaN,NaN,NaN
588,"[PBSC Urban Solutions, Shift Transit]",/v2/networks/tugo-bike-share,tugo-bike-share,Tugo Bike Share,"Tucson, AZ",US,32.222500,-110.974900,NaN,https://tucson.publicbikesystem.net/customer/g...,NaN,NaN,NaN


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

## Step 3: Parse Main Directory for Target City

* Filter the list of network providers for one that is in the target city. 
* declare a variable for 'provider id' based on filter result
* use this variable the argument into a second API call to get the specific details on the provider. (overrides the default "")


In [5]:
cities = city_bike_df[city_bike_df['location.city']=='Vancouver']
city_ids = cities['id']
city_ids 
    

210    mobibikes
Name: id, dtype: object

# Step 4: Use Provider ID to return detailed information. 
* use existing API call from above, but amend with the [city_id] in the URL suffix.
* Since there is only 1 provider in the target city, run the function as a standalone. 
* If there were multiple results, would nest it into a FOR LOOP to generate multiple api calls. 

In [6]:
# Use the Provider ID from above as the arguement for city_bike_api_get to receive the details on the provider. 
provider_id ='mobibikes'
url = f"http://api.citybik.es/v2/networks/{provider_id}"
payload = {}
headers = {}
print(url)

response_mobibikes = requests.request("GET", url, headers=headers, data=payload)

# print(response.text)

if response.status_code == 200:
        # Construct the filename with the index
        filename = f'output_{provider_id}.json'
        
        # Assuming you want to save the data as 'output....json'
        with open(filename, 'w') as json_file:
            json.dump(response_mobibikes.json(), json_file, indent=4)
            print('JSON data saved successfully.')
else:
    print(f"Error: Received status code {response.status_code}")

vancouver_json= response_mobibikes.json()


http://api.citybik.es/v2/networks/mobibikes
JSON data saved successfully.


Put your parsed results into a DataFrame.

In [31]:
# # Use pd.json_normalize to flatten the JSON array
vancouver_df = pd.json_normalize(vancouver_json['network']['stations'])
vancouver_df.info()
vancouver_df


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   empty_slots         245 non-null    int64  
 1   free_bikes          245 non-null    int64  
 2   id                  245 non-null    object 
 3   latitude            245 non-null    float64
 4   longitude           245 non-null    float64
 5   name                245 non-null    object 
 6   timestamp           245 non-null    object 
 7   extra.ebikes        245 non-null    int64  
 8   extra.has_ebikes    245 non-null    bool   
 9   extra.last_updated  245 non-null    int64  
 10  extra.normal_bikes  245 non-null    int64  
 11  extra.renting       245 non-null    bool   
 12  extra.returning     245 non-null    bool   
 13  extra.slots         245 non-null    int64  
 14  extra.uid           245 non-null    object 
dtypes: bool(3), float64(2), int64(6), object(4)
memory usage:

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.ebikes,extra.has_ebikes,extra.last_updated,extra.normal_bikes,extra.renting,extra.returning,extra.slots,extra.uid
0,16,18,7a19c49f486d7c0c02b3685d7b240448,49.262487,-123.114397,10th & Cambie,2023-09-21T15:03:45.749000Z,2,True,1695308531,16,True,True,36,0001
1,1,15,32603a87cfca71d0f7dfa3513bad69d5,49.274566,-123.121817,Yaletown-Roundhouse Station,2023-09-21T15:03:45.745000Z,2,True,1695308550,13,True,True,16,0004
2,12,14,6d42fa40360f9a6b2bf641c7b8bb2862,49.279764,-123.110154,Dunsmuir & Beatty,2023-09-21T15:03:45.746000Z,1,True,1695308228,13,True,True,26,0005
3,12,4,66f873d641d448bd1572ab086665a458,49.260599,-123.113504,12th & Yukon (City Hall),2023-09-21T15:03:45.747000Z,4,True,1695308356,0,True,True,16,0007
4,11,5,485d4d24c803cfde829ab89699fed833,49.264215,-123.117772,8th & Ash,2023-09-21T15:03:45.761000Z,3,True,1695308529,2,True,True,16,0008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,33,7,a74744ce4bb7ea2aa9f406ac8bff95d8,49.280977,-123.035969,PNE - Hastings & Windermere,2023-09-21T15:03:45.669000Z,2,True,1695308314,5,True,True,40,1002
241,10,9,5699b40126e10c2f68eefc3eb18ff3a1,49.270783,-123.141564,1st & Fir,2023-09-21T15:03:45.649000Z,2,True,1695308537,7,True,True,20,0315
242,17,5,34fd37d12eb989b49518ef53941ee3ff,49.264019,-123.209176,Sasamat & 10th,2023-09-21T15:03:45.576000Z,1,True,1695308185,4,True,True,22,0716
243,18,2,b7dd37a7dd668d6d10024b7f18acc438,49.265800,-123.205960,Trimble & 8th,2023-09-21T15:03:45.646000Z,1,True,1695308518,1,True,True,20,0715


## Step 6: Export Dataframe to Postgres
* identify which data types are present the the City bike data frame
* Map them to appropriate fields for SQL
* Setup the connection string and functions to create/append DB


In [13]:
vancouver_df.dtypes

empty_slots             int64
free_bikes              int64
id                     object
latitude              float64
longitude             float64
name                   object
timestamp              object
extra.ebikes            int64
extra.has_ebikes         bool
extra.last_updated      int64
extra.normal_bikes      int64
extra.renting            bool
extra.returning          bool
extra.slots             int64
extra.uid              object
dtype: object

In [8]:
### QUERY - CREATE the CITY_BIKES TABLE

create_citybikes_tble = """
CREATE TABLE IF NOT EXISTS city_bikes (
    empty_slots integer,
    free_bikes integer,
    id text,
    latitude double precision,
    longitude double precision,
    name text,
    timestamp timestamp,
    extra_ebikes integer,
    extra_has_ebikes boolean,
    extra_last_updated bigint
    ); 
"""

In [2]:
import psycopg2

In [14]:
### DEFINE CONNECTION STRING TO POSTGRES
# Define the connection parameters
db_params = {
        'host': '192.168.0.21',       # PostgreSQL host on ubuntu
        'port': '5432',       # Default port
        'database': 'LHL-Statistics-Project',     # Database name
        'user': 'jamie',   # obvs
        'password': 'b3laf0nt3' # obvs
    }

def create_connection(db_params):
    # Create a connection to the PostgreSQL database
    try:
        connection = psycopg2.connect(**db_params)
        print("Connected to PostgreSQL database!")     
           
    except psycopg2.Error as e:
        print("Error connecting to PostgreSQL:", e)

## Create a CONNECTION between python and the sqlite file 'sm_app'
connection = create_connection(db_params)

Connected to PostgreSQL database!


In [9]:
### DEFINE FUNCTION TO EXECUTE QUERIES

import psycopg2
from psycopg2 import Error

def execute_structure_qry(connection, query):
    try:
        connection = psycopg2.connect(**db_params)
        print("Connected to PostgreSQL database!")
        try:
            cursor = connection.cursor()
            cursor.execute(query)
            cursor.close()
            connection.commit()
            print("Query executed successfully")
        except Error as e:
            connection.rollback()
            print(f"Error executing query: {e}")
            
    except psycopg2.Error as e:
        print("Error connecting to PostgreSQL:", e)

   # Close the connection when done
    connection.close()

execute_structure_qry(connection,create_citybikes_tble)



Connected to PostgreSQL database!
Query executed successfully


In [21]:
### DEFINE SELECT or READ  QUERY FUNCTION

def execute_read_query(connection, query):
    try:
        connection = psycopg2.connect(**db_params)
        print("Connected to PostgreSQL database!")
    
        cursor = connection.cursor()
        result = None
        try:
            cursor.execute(query)
            result = cursor.fetchall()
            return result
        except Error as e:
            print(f"The error '{e}' occurred")
    except psycopg2.Error as e:
        print("Error connecting to PostgreSQL:", e)



In [22]:
## DEFINE SELECT QUERIES
test_query = "SELECT * from city_bikes;"


In [23]:
import psycopg2
from psycopg2 import Error

## CALL SELECT QUERIES
## Create a CONNECTION between python and the sqlite file 'sm_app'

execute_read_query(connection,test_query)

Connected to PostgreSQL database!


[]